In [1]:
%load_ext autoreload
%autoreload 2

Set up the gym and wrap a monitor around it that will periodically record movies as it learns

In [2]:
from surface_seg.envs.multicomponent_surface_env import MultiComponentSurface
import gym.wrappers

MCS_gym = MultiComponentSurface()
MCS_gym = gym.wrappers.Monitor(MCS_gym, "./vid", force=True)

print('Starting energy = %1.2f eV'%MCS_gym.atoms.get_potential_energy())

Starting energy = 2.49 eV


/global/homes/z/zulissi/miniconda3/envs/surface_seg/lib/python3.8/site-packages/gym-0.17.2-py3.8.egg/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Set up the gym and agent in tensorforce

In [3]:
import tensorforce 
from tensorforce.agents import Agent


env = tensorforce.environments.OpenAIGym(MCS_gym,
                                                 max_episode_timesteps=200,
                                                visualize=False)

agent = Agent.create(
    agent='dqn', 
    environment=env, 
    batch_size=10, 
    learning_rate=1e-3,
    memory = 40000,
    max_episode_timesteps = 200,
    exploration=0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Run the DRL method for a while

In [ ]:
from tensorforce.execution import Runner

runner = Runner(
    agent=agent,
    environment=env,
    max_episode_timesteps=200
)

runner.run(num_episodes=1000)

# runner.run(num_episodes=10, evaluation=True)

runner.close()

Episodes:   2%|▏         | 24/1000 [04:29, reward=-0.65, ts/ep=200, sec/ep=10.00, ms/ts=50.0, agent=9.0%]

# Example of how to render the environment in a jupyter cell

In [ ]:
env = MCS_gym
from matplotlib import pyplot as plt
%matplotlib inline

from IPython import display

img = plt.imshow(env.render(mode='rgb_array')) # only call this once
plt.axis('off')
for _ in range(100):
    img.set_data(env.render(mode='rgb_array')) # just update the data
    display.display(plt.gcf())
    display.clear_output(wait=True)
    action = env.action_space.sample()
    env.step(action)